In [ ]:
import logging
from pathlib import Path
import time
from sandbox.virtualmachine import AgentVMManager, AgentVMConfig

# Adjust import path if needed

# Setup logging to see what’s happening
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("NotebookAgentVM")

# Prepare config
cfg = AgentVMConfig(
    container_name="test-agent-vm",
    host_server_dir=Path("server/"),  # Your server directory with start.sh and main.py
    host_sandbox_server_port=8765,
    host_sandbox_server_host="localhost",
    sandbox_server_port=8765,
    sandbox_server_host="0.0.0.0",
    sandbox_server_dir=Path("/home/user/server"),
    # unique_container_name=True,
)

# Start the agent VM and check if the WebSocket server is ready
try:
    logger.info("🚀 Starting Agent VM and waiting for FastAPI server...")
    with AgentVMManager(cfg, logger=logger) as vm:
        logger.info("✅ Sandbox VM started!")

        # logger.info(vm.tail_server_logs(20))

        time.sleep(360)

except Exception as e:
    logger.error(f"\n❌ Error during startup: {e}")
